In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import numpy as np
import mysql.connector


In [2]:
df_sal=pd.read_csv("Sales.csv")
df_sal

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [3]:
df_sal.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [4]:
df_sal['Delivery Date']=pd.to_datetime(df_sal['Delivery Date'], format="%m/%d/%Y",errors='coerce')

In [5]:
df_sal['Delivery Date']=df_sal['Delivery Date'].fillna(0)

In [6]:
df_sal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order Number   62884 non-null  int64 
 1   Line Item      62884 non-null  int64 
 2   Order Date     62884 non-null  object
 3   Delivery Date  62884 non-null  object
 4   CustomerKey    62884 non-null  int64 
 5   StoreKey       62884 non-null  int64 
 6   ProductKey     62884 non-null  int64 
 7   Quantity       62884 non-null  int64 
 8   Currency Code  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [7]:
df_exc=pd.read_csv("Exchange_Rates.csv")
df_exc

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415
...,...,...,...
11210,2/20/2021,USD,1.0000
11211,2/20/2021,CAD,1.2610
11212,2/20/2021,AUD,1.2723
11213,2/20/2021,EUR,0.8238


In [8]:
df_exc['Date']=pd.to_datetime(df_exc['Date'], format="%m/%d/%Y",errors='coerce')

In [9]:
df_exc

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415
...,...,...,...
11210,2021-02-20,USD,1.0000
11211,2021-02-20,CAD,1.2610
11212,2021-02-20,AUD,1.2723
11213,2021-02-20,EUR,0.8238


In [10]:
df_sto=pd.read_csv("Stores.csv")
df_sto

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [11]:
df_sto=df_sto.iloc[:-1]
df_sto

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
61,62,United States,South Dakota,1120.0,6/3/2018
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012


In [12]:
df_sal.isnull().sum()

Order Number     0
Line Item        0
Order Date       0
Delivery Date    0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency Code    0
dtype: int64

In [13]:
df_prod=pd.read_csv("Products.csv")
df_prod

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones


In [14]:
df_prod["Unit Cost USD"]=df_prod["Unit Cost USD"].str.replace("$","").str.replace(",","")
df_prod["Unit Cost USD"]=df_prod["Unit Cost USD"].astype(float)
df_prod["Unit Price USD"]=df_prod["Unit Price USD"].str.replace("$","").str.replace(",","")
df_prod["Unit Price USD"]=df_prod["Unit Price USD"].astype(float)


In [15]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ProductKey      2517 non-null   int64  
 1   Product Name    2517 non-null   object 
 2   Brand           2517 non-null   object 
 3   Color           2517 non-null   object 
 4   Unit Cost USD   2517 non-null   float64
 5   Unit Price USD  2517 non-null   float64
 6   SubcategoryKey  2517 non-null   int64  
 7   Subcategory     2517 non-null   object 
 8   CategoryKey     2517 non-null   int64  
 9   Category        2517 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 196.8+ KB


In [16]:
df_cus=pd.read_csv("Customers.csv",encoding='ISO-8859-1')
df_cus

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


In [17]:
df_cus['Birthday']=pd.to_datetime(df_cus['Birthday'], format="%m/%d/%Y",errors='coerce').dt.date

In [18]:
df_cus.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [19]:
df_cus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB


In [20]:
df_cus.drop(["State Code","Zip Code"],axis=1,inplace=True)

In [21]:
df_sal.isnull().sum()


Order Number     0
Line Item        0
Order Date       0
Delivery Date    0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency Code    0
dtype: int64

In [22]:
df_cus.isnull().sum()


CustomerKey    0
Gender         0
Name           0
City           0
State          0
Country        0
Continent      0
Birthday       0
dtype: int64

In [23]:
df_exc.isnull().sum()


Date        0
Currency    0
Exchange    0
dtype: int64

In [24]:
df_sto.isnull().sum()


StoreKey         0
Country          0
State            0
Square Meters    0
Open Date        0
dtype: int64

In [25]:
df_prod.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [26]:
conn=mysql.connector.connect(host="localhost",user="root",password="vision@123",database="dataspark")
my_cursor=conn.cursor()



In [27]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS customer_details(
                  CustomerKey INT,
                  Gender VARCHAR(255),
                  Name VARCHAR(255),
                  City VARCHAR(255),
                  State VARCHAR(255),
                  Country VARCHAR(255),
                  Continent VARCHAR(255),
                  Birthday VARCHAR(255)
                  )''')
print("Success")


Success


In [28]:
ins_que='''INSERT INTO customer_details(
                  CustomerKey,
                  Gender,
                  Name,
                  City,
                  State,
                  Country,
                  Continent,
                  Birthday)
                  VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'''
data=df_cus.values.tolist()
my_cursor.executemany(ins_que,data)

print("Inserted")

Inserted


In [29]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS exchage_details(
                  Date VARCHAR(255),
                  Currency VARCHAR(255),
                  Exchange FLOAT
                  )''')
print("Table Created")

Table Created


In [31]:
ins_que='''INSERT INTO exchage_details(
                  Date,
                  Currency,
                  Exchange)
                  VALUES(%s, %s, %s)'''
data=df_exc.values.tolist()
my_cursor.executemany(ins_que,data)

print("Inserted")

Inserted


In [42]:
df_exc

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415
...,...,...,...
11210,2021-02-20,USD,1.0000
11211,2021-02-20,CAD,1.2610
11212,2021-02-20,AUD,1.2723
11213,2021-02-20,EUR,0.8238


In [60]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS products_table(
                  
                  ProductKey INT,
                  ProductName VARCHAR(255),
                  Brand VARCHAR(255),
                  Color VARCHAR(255),
                  Unit_Cost_USD FLOAT,
                  Unit_Price_USD FLOAT,
                  SubcategoryKey INT,
                  Subcategory VARCHAR(255),
                  CategoryKey INT,
                  Category VARCHAR(255)
                  )''')
print("Success")


Success


In [61]:
ins_que='''INSERT INTO products_table(
                  
                  ProductKey,
                  ProductName,
                  Brand,
                  Color,
                  Unit_Cost_USD,
                  Unit_Price_USD,
                  SubcategoryKey,
                  Subcategory,
                  CategoryKey,
                  Category)
                  VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
data=df_prod.values.tolist()
my_cursor.executemany(ins_que,data)

print("Inserted")

Inserted


In [55]:
data

[[1,
  'Contoso 512MB MP3 Player E51 Silver',
  'Contoso',
  'Silver',
  6.62,
  12.99,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [2,
  'Contoso 512MB MP3 Player E51 Blue',
  'Contoso',
  'Blue',
  6.62,
  12.99,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [3,
  'Contoso 1G MP3 Player E100 White',
  'Contoso',
  'White',
  7.4,
  14.52,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [4,
  'Contoso 2G MP3 Player E200 Silver',
  'Contoso',
  'Silver',
  11.0,
  21.57,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [5,
  'Contoso 2G MP3 Player E200 Red',
  'Contoso',
  'Red',
  11.0,
  21.57,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [6,
  'Contoso 2G MP3 Player E200 Black',
  'Contoso',
  'Black',
  11.0,
  21.57,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [7,
  'Contoso 2G MP3 Player E200 Blue',
  'Contoso',
  'Blue',
  11.0,
  21.57,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [8,
  'Contoso 4G MP3 Player E400 Silver',
  'Contoso',
  'Silver',
  30.58,
  59.99,
  101,
  'MP4&MP3',
  1,
  'Audio'],
 [9,
  'Contoso 4G MP3 Player 

In [50]:
df_prod

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [51]:
col=list(df_prod.columns)
col1=[]
for c in col:
    col1.append(c.replace(' ','_').lower())
df_prod.columns=col1

In [54]:
df_prod

,productkey,product_name,brand,color,unit_cost_usd,unit_price_usd,subcategorykey,subcategory,categorykey,category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [38]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS sales_details(
                  Order_Number INT,
                  Line_Item INT,
                  Order_Date VARCHAR(255),
                  Delivery_Date VARCHAR(255),
                  CustomerKey INT,
                  StoreKey INT,
                  ProductKey INT,
                  Quantity INT,
                  Currency_Code VARCHAR(255)
                  )''')
print("Success")


Success


In [1]:
df_sal

NameError: name 'df_sal' is not defined

In [43]:
ins_que='''INSERT INTO sales_details(
                  Order_Number,
                  Line_Item,
                  Order_Date,
                  Delivery_Date,
                  CustomerKey,
                  StoreKey,
                  ProductKey,
                  Quantity,
                  Currency_Code)
                  VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
data=df_sal.values.tolist()
my_cursor.executemany(ins_que,data)

print("Inserted")

Inserted


In [40]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS store_details(
                  StoreKey INT,
                  Country VARCHAR(255),
                  State VARCHAR(255),
                  Square_Metres FLOAT,
                  Open_Date VARCHAR(255)
                  )''')
print("Success")


Success


In [46]:
ins_que='''INSERT INTO store_details(
                  StoreKey,
                  Country,
                  State,
                  Square_Metres,
                  Open_Date)
                  VALUES(%s, %s, %s, %s, %s)'''
data=df_sto.values.tolist()
my_cursor.executemany(ins_que,data)

print("Inserted")

Inserted
